In [ ]:
import pandas   as pd
import polars   as pl
import numpy    as np
import networkx as nx
from math import cos, sin, pi, sqrt, atan2, floor, ceil
import random
import rtsvg
rt      = rtsvg.RACETrack()
relates = [('fm','to')]

# Read in the simple graph templates
df      = pl.read_csv('../rtsvg/config/simple_graph_df.csv')
g       = rt.createNetworkXGraph(df, relates)

# Create the component lookup structure
comp_lu = {}
for _node_set_ in nx.connected_components(g):
    _g_              = g.subgraph(_node_set_)
    _nodes_, _edges_ = _g_.number_of_nodes(), _g_.number_of_edges()
    if _nodes_ not in comp_lu:          comp_lu[_nodes_]          = {}
    if _edges_ not in comp_lu[_nodes_]: comp_lu[_nodes_][_edges_] = []
    comp_lu[_nodes_][_edges_].append(_g_)

# Positional Information
_df_    = pd.read_csv('../rtsvg/config/simple_graph_layouts.csv')
pos_templates  = {}
for i in range(len(_df_)): pos_templates[_df_['node'][i]] = [_df_['x'][i], _df_['y'][i]]

# Sanity Check
set(df['fm']) | set(df['to']) == set(pos_templates.keys())

In [2]:
#pos = rt.treeMapGraphComponentPlacement(g, pos_templates, bounds_percent=0.3)
#rt.link(df, relates, pos, w=1200,h=900)

In [ ]:
#
#
#
def graphRemoveAllOneDegreeNodes(self, _g_):
    to_remove, removed_nodes = [], {}
    for _node_ in _g_.nodes():
        if _g_.degree(_node_) == 1:
            to_remove.append(_node_)
            _still_in_ = list(_g_.neighbors(_node_))[0]
            if _still_in_ not in removed_nodes: removed_nodes[_still_in_] = set()
            removed_nodes[_still_in_].add(_node_)
    g_after_removal = _g_.copy()
    g_after_removal.remove_nodes_from(to_remove)
    return g_after_removal, removed_nodes

#
#
#
def __oneDegreeNodes_clouds__(self, g_after_removal, removed_nodes, pos, degree_one_method):
    for _node_ in removed_nodes.keys():
        # Determine the minimum distance to a neighbor
        min_distance_to_nbor = 1e9
        for _nbor_ in g_after_removal.neighbors(_node_):
            d = self.segmentLength((pos[_node_], pos[_nbor_]))
            if d < min_distance_to_nbor: min_distance_to_nbor = d
        if min_distance_to_nbor == 1e9: min_distance_to_nbor = 1.0
        # Determine the average angle *AWAY* from all the other nodes (on average)
        uv_sum, uv_samples = (0.0, 0.0), 0
        for _others_ in g_after_removal.nodes():
            if _others_ != _node_:
                uv         = self.unitVector((pos[_others_], pos[_node_]))
                uv_sum     = (uv_sum[0] + uv[0], uv_sum[1] + uv[1])
                uv_samples += 1
        if uv_samples > 0: uv = (uv_sum[0] / uv_samples, uv_sum[1] / uv_samples)
        else:              uv = (1.0, 0.0)
        # Apply the different methods
        if degree_one_method == 'clouds_sunflower':
            _xy_ = (pos[_node_][0] + uv[0] * min_distance_to_nbor/4.0, pos[_node_][1] + uv[1] * min_distance_to_nbor/4.0)
            self.sunflowerSeedArrangement(g_after_removal, removed_nodes[_node_], pos, _xy_, min_distance_to_nbor/8.0)
        else:
            for _removed_ in removed_nodes[_node_]:
                pos[_removed_] = (pos[_node_][0] + uv[0] * min_distance_to_nbor/4.0, pos[_node_][1] + uv[1] * min_distance_to_nbor/4.0)

#
#
#
def __oneDegreeNodes_circular__(self, g_after_removal, removed_nodes, pos, buffer_in_degrees=30.0):
    for _node_ in removed_nodes.keys():
        # Determine the minimum distance to a neighbor
        min_distance_to_nbor = 1e9
        for _nbor_ in g_after_removal.neighbors(_node_):
            d = self.segmentLength((pos[_node_], pos[_nbor_]))
            if d < min_distance_to_nbor: min_distance_to_nbor = d
        if min_distance_to_nbor == 1e9: min_distance_to_nbor = 1.0

        # Calculate the angle buffers
        num_of_pts, angle_buffer, r = len(removed_nodes[_node_]), 2.0 * pi * buffer_in_degrees/360.0, min_distance_to_nbor/4.0
        _xy_   = pos[_node_]
        angles = []
        for _nbor_ in g_after_removal.neighbors(_node_):
            _nbor_xy_ = pos[_nbor_]
            if _nbor_xy_ != _xy_: uv = rt.unitVector((_xy_, _nbor_xy_))
            else:              uv = (1.0, 0.0)
            _angle_ = atan2(uv[1], uv[0])
            if _angle_ < 0: _angle_ += 2*pi
            angles.append(_angle_)
        angles         = sorted(angles)
        cleared_angles, circumference_sum = [], 0.0
        for i in range(len(angles)):
            _a0_, _a1_ = angles[i], angles[(i+1) % len(angles)]
            if i == len(angles)-1: _a1_ += 2*pi
            _a_diff_ = _a1_ - _a0_ - 2*angle_buffer
            if _a_diff_ > angle_buffer:
                _a0_buffered_     = _a0_ + angle_buffer
                _a1_buffered_     = _a1_ - angle_buffer
                _circumference_   = 2.0 * pi * r * (_a1_buffered_ - _a0_buffered_)/(2*pi)
                cleared_angles.append((_a0_buffered_, _a1_buffered_, _circumference_))
                circumference_sum += _circumference_

        nodes_to_plot = list(removed_nodes[_node_])
        if circumference_sum > 0.0:
            # Allocate points to each of the arc segments
            pts_w_arcs, _pts_left_ = [], num_of_pts
            for i in range(len(cleared_angles)):
                _a0_, _a1_, _circ_  = cleared_angles[i]
                if i == len(cleared_angles)-1: _pts_to_allocate_ = _pts_left_
                else:                          _pts_to_allocate_ = int(num_of_pts * _circ_ / circumference_sum)
                _pts_left_ -= _pts_to_allocate_
                pts_w_arcs.append(_pts_to_allocate_)

            # Plot the points
            node_i = 0
            for i in range(len(cleared_angles)):
                _a0_, _a1_, _circ_ = cleared_angles[i]
                pts_on_segment = pts_w_arcs[i]
                if   pts_on_segment == 1:
                    _angle_ = (_a0_ + _a1_) / 2.0
                    pos[nodes_to_plot[node_i]] = (_xy_[0]+r*cos(_angle_), _xy_[1]+r*sin(_angle_))
                    node_i += 1
                elif pts_on_segment == 2:
                    _angle_ = (_a0_ + _a1_) / 2.0 - (_a0_ - _a1_) / 4.0
                    pos[nodes_to_plot[node_i]] = (_xy_[0]+r*cos(_angle_), _xy_[1]+r*sin(_angle_))
                    node_i += 1
                    _angle_ = (_a0_ + _a1_) / 2.0 + (_a0_ - _a1_) / 4.0
                    pos[nodes_to_plot[node_i]] = (_xy_[0]+r*cos(_angle_), _xy_[1]+r*sin(_angle_))
                    node_i += 1
                else:
                    _angle_inc_ = (_a1_ - _a0_) / (pts_on_segment - 1)
                    for j in range(pts_on_segment):
                        _angle_ = _a0_ + _angle_inc_ * j
                        pos[nodes_to_plot[node_i]] = (_xy_[0]+r*cos(_angle_), _xy_[1]+r*sin(_angle_))
                        node_i += 1

        else: # just dump them around the circle
            _angle_inc_ = 2.0 * pi / num_of_pts
            for i in range(num_of_pts): pos[nodes_to_plot[i]] = (_xy_[0]+r*cos(_angle_inc_*i), _xy_[1]+r*sin(_angle_inc_*i))

#
#
#
def layoutSimpleTemplates(self, g, pos, degree_one_method='clouds'):
    # Validate input parameters
    _methods_ = {'clouds', 'clouds_sunflower', 'circular'}
    if degree_one_method not in _methods_: raise ValueError(f'Invalid degree one method: {degree_one_method} -- accepted methods: {_methods_}')
    # For each connected component
    for _node_set_ in nx.connected_components(g):
        _g_              = g.subgraph(_node_set_)
        _nodes_, _edges_ = _g_.number_of_nodes(), _g_.number_of_edges()
        match_found      = False
        if _nodes_ in comp_lu and _edges_ in comp_lu[_nodes_]:
            for _g_template_ in comp_lu[_nodes_][_edges_]:
                if nx.is_isomorphic(_g_, _g_template_):
                    # If pattern matches, copy the template over
                    gm     = nx.isomorphism.GraphMatcher(_g_, _g_template_)
                    _dict_ = next(gm.subgraph_isomorphisms_iter())
                    for k in _dict_.keys(): pos[k] = pos_templates[_dict_[k]]
                    match_found = True
                    break
        # if no match was found, try the pattern matching with one degree nodes removed
        if not match_found:
            g_after_removal, removed_nodes = graphRemoveAllOneDegreeNodes(self, _g_)
            _nodes_, _edges_ = g_after_removal.number_of_nodes(), g_after_removal.number_of_edges()
            if _nodes_ in comp_lu and _edges_ in comp_lu[_nodes_]:
                for _g_template_ in comp_lu[_nodes_][_edges_]:
                    if nx.is_isomorphic(g_after_removal, _g_template_):
                        # If pattern matches, copy the template over
                        gm     = nx.isomorphism.GraphMatcher(g_after_removal, _g_template_)
                        _dict_ = next(gm.subgraph_isomorphisms_iter())
                        for k in _dict_.keys(): pos[k] = pos_templates[_dict_[k]]
                        # Add the one degrees back in
                        if   degree_one_method == 'clouds' or degree_one_method == 'clouds_sunflower': __oneDegreeNodes_clouds__  (self, g_after_removal, removed_nodes, pos, degree_one_method)
                        elif degree_one_method == 'circular':                                          __oneDegreeNodes_circular__(self, g_after_removal, removed_nodes, pos)
                        match_found = True
                        break
            elif _nodes_ == 1: # star pattern
                _node_ = list(g_after_removal.nodes())[0]
                pos[_node_] = (0.0, 0.0)
                if   len(removed_nodes[_node_]) < 10:
                    x, y, y_inc = 1.0, -0.5, 1.0/(len(removed_nodes[_node_])-1)
                    for _removed_ in removed_nodes[_node_]:
                        pos[_removed_] = (x, y)
                        y += y_inc
                elif len(removed_nodes[_node_]) < 40:
                    _angle_inc_ = 2.0 * pi / len(removed_nodes[_node_])
                    _angle_     = 0.0
                    for _removed_ in removed_nodes[_node_]:
                        pos[_removed_] = (cos(_angle_), sin(_angle_))
                        _angle_ += _angle_inc_
                else:
                    self.sunflowerSeedArrangement(g_after_removal, removed_nodes[_node_], pos, (1.0, 0.0), 0.5)


    # finally, organize using a treemap scaled by the number of nodes
    return self.treeMapGraphComponentPlacement(g, pos, bounds_percent=0.3)

_lu_ = {'src':['a','b','c','d','e','f', 'w', 'x', 'y', 'z', 'solo'],
        'dst':['b','c','a','e','f','d', 'x', 'y', 'z', 'w', 'solo']}
for i in range(random.randint(10,20)):
    if random.random() > 0.5: _lu_['src'].append(f'a{i}'), _lu_['dst'].append(f'a')
    else:                     _lu_['dst'].append(f'a{i}'), _lu_['src'].append(f'a')
for i in range(random.randint(20,30)):
    if random.random() > 0.5: _lu_['src'].append(f'b{i}'), _lu_['dst'].append(f'b')
    else:                     _lu_['dst'].append(f'b{i}'), _lu_['src'].append(f'b')
for i in range(random.randint(30,40)):
    if random.random() > 0.5: _lu_['src'].append(f'c{i}'), _lu_['dst'].append(f'c')
    else:                     _lu_['dst'].append(f'c{i}'), _lu_['src'].append(f'c')
for i in range(random.randint(20,30)):
    if random.random() > 0.5: _lu_['src'].append(f'solo{i}'),  _lu_['dst'].append(f'solo')
    else:                     _lu_['dst'].append(f'solo{i}'),  _lu_['src'].append(f'solo')
for i in range(random.randint(200,300)):
    if random.random() > 0.5: _lu_['src'].append(f'again{i}'), _lu_['dst'].append(f'again')
    else:                     _lu_['dst'].append(f'again{i}'), _lu_['src'].append(f'again')
for i in range(random.randint(10,30)):
    _lu_['src'].append('y'), _lu_['dst'].append(f'y_{i}')
df2   = pl.DataFrame(_lu_)
g_df2 = rt.createNetworkXGraph(df2, [('src','dst')])
pos   = {}
for _node_ in g_df2.nodes(): pos[_node_] = (random.random(), random.random())
rt.tile([rt.link(df2, [('src','dst')], layoutSimpleTemplates(rt, g_df2, pos, 'clouds'),           node_size='small', w=384, h=384),
         rt.link(df2, [('src','dst')], layoutSimpleTemplates(rt, g_df2, pos, 'clouds_sunflower'), node_size='small', w=384, h=384),
         rt.link(df2, [('src','dst')], layoutSimpleTemplates(rt, g_df2, pos, 'circular'),         node_size='small', w=384, h=384)], spacer=10)

In [ ]:
rt.tile([rt.link(df2, [('src','dst')], rt.layoutSimpleTemplates(g_df2, pos, 'clouds'),           node_size='small', w=384, h=384),
         rt.link(df2, [('src','dst')], rt.layoutSimpleTemplates(g_df2, pos, 'clouds_sunflower'), node_size='small', w=384, h=384),
         rt.link(df2, [('src','dst')], rt.layoutSimpleTemplates(g_df2, pos, 'circular'),         node_size='small', w=384, h=384)], spacer=10)

In [ ]:
#
# Circular Option -- this version has off by one errors... and is just poorly written
#

from math import atan2, ceil, floor

svg_hdr = '<svg x="0" y="0" width="512" height="512" viewbox="0 0 256 256"><rect x="0" y="0" width="256" height="256" fill="#ffffff" />'
svg     = []
_xy_    = (100.5, 100.5)
_nbors_ = []
for i in range(4): _nbors_.append((random.randint(0,255), random.randint(0,255)))
#_nbors_  = [(0, 100.5), (40,200.5)]
#_nbors_ = [(100.5, 0.0),(0, 100.5)]
num_of_pts, angle_buffer, r = 60, 2.0 * pi * 15.0/360.0, 50.0

for _nbor_ in _nbors_: svg.append(f'<line x1="{_xy_[0]}" y1="{_xy_[1]}" x2="{_nbor_[0]}" y2="{_nbor_[1]}" stroke="black" stroke-width="2" />')
svg.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="{r}" fill="none" stroke="black" stroke-width="0.4" />')
angles = []
for _nbor_ in _nbors_:
    if _nbor_ != _xy_: uv = rt.unitVector((_xy_, _nbor_))
    else:              uv = (1.0, 0.0)
    _angle_ = atan2(uv[1], uv[0])
    if _angle_ < 0: _angle_ += 2*pi
    angles.append(_angle_)
angles         = sorted(angles)
cleared_angles, circumference_sum = [], 0.0
for i in range(len(angles)):
    _a0_, _a1_ = angles[i], angles[(i+1) % len(angles)]
    if i == len(angles)-1: _a1_ += 2*pi
    _a_diff_ = _a1_ - _a0_ - 2*angle_buffer
    if _a_diff_ > angle_buffer:
        _a0_buffered_     = _a0_ + angle_buffer
        _a1_buffered_     = _a1_ - angle_buffer
        _circumference_   = 2.0 * pi * r * (_a1_buffered_ - _a0_buffered_)/(2*pi)
        cleared_angles.append((_a0_buffered_, _a1_buffered_, _circumference_))
        circumference_sum += _circumference_

if circumference_sum > 0.0:
    pts_left, pts_plotted = num_of_pts, 0 
    for i in range(len(cleared_angles)):
        _a0_, _a1_, _circ_ = cleared_angles[i]
        pts_on_segment = floor(num_of_pts * _circ_ / circumference_sum)
        if i == len(cleared_angles)-1: pts_on_segment = pts_left
        if pts_on_segment > 1:
            _angle_inc_ = (_a1_ - _a0_) / (pts_on_segment - 1)
            while _a0_ <= _a1_:
                if pts_plotted > num_of_pts: break
                svg.append(f'<circle cx="{_xy_[0]+r*cos(_a0_)}" cy="{_xy_[1]+r*sin(_a0_)}" r="2" fill="#ff0000" stroke="none" />')
                pts_plotted += 1
                _a0_ += _angle_inc_
        elif pts_on_segment == 1:
            if pts_plotted > num_of_pts: break
            svg.append(f'<circle cx="{_xy_[0]+r*cos((_a0_ + _a1_)/2.0)}" cy="{_xy_[1]+r*sin((_a0_ + _a1_)/2.0)}" r="5" fill="#ff0000" stroke="none" />')
            pts_plotted += 1
        pts_left -= pts_on_segment
    print(f'{pts_left=} {num_of_pts=} {pts_plotted=}')
else: # just dump them around the circle
    _angle_inc_ = 2.0 * pi / num_of_pts
    for i in range(num_of_pts):
        svg.append(f'<circle cx="{_xy_[0]+r*cos(_angle_inc_*i)}" cy="{_xy_[1]+r*sin(_angle_inc_*i)}" r="2" fill="#ff0000" stroke="none" />')

def radToDeg(_rad_): return _rad_ * 180.0 / pi
for _arc_ in cleared_angles:
    _a0_, _a1_, _circ_ = radToDeg(_arc_[0]), radToDeg(_arc_[1]), _arc_[2]
    svg.append(f'<path d=\"{rt.genericArc(_xy_[0], _xy_[1], _a0_, _a1_, r-5, r+5)}\" fill=\"#000000\" fill-opacity="0.1" stroke=\"black\" stroke-width=\"0.2\" />')

svg_ftr = '</svg>'
rt.tile([svg_hdr+''.join(svg)+svg_ftr])

In [ ]:
#
# Circular Version #2 - this version has no off by one errors ... but is still poorly written
#

svg_hdr = '<svg x="0" y="0" width="512" height="512" viewbox="0 0 256 256"><rect x="0" y="0" width="256" height="256" fill="#ffffff" />'
svg     = []
_xy_    = (100.5, 100.5)
_nbors_ = []
for i in range(4): _nbors_.append((random.randint(0,255), random.randint(0,255)))
#_nbors_  = [(0, 100.5), (40,200.5)]
#_nbors_ = [(100.5, 0.0),(0, 100.5)]
num_of_pts, angle_buffer, r = 20, 2.0 * pi * 15.0/360.0, 50.0

for _nbor_ in _nbors_: svg.append(f'<line x1="{_xy_[0]}" y1="{_xy_[1]}" x2="{_nbor_[0]}" y2="{_nbor_[1]}" stroke="black" stroke-width="2" />')
svg.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="{r}" fill="none" stroke="black" stroke-width="0.4" />')
angles = []
for _nbor_ in _nbors_:
    if _nbor_ != _xy_: uv = rt.unitVector((_xy_, _nbor_))
    else:              uv = (1.0, 0.0)
    _angle_ = atan2(uv[1], uv[0])
    if _angle_ < 0: _angle_ += 2*pi
    angles.append(_angle_)
angles         = sorted(angles)
cleared_angles, circumference_sum = [], 0.0
for i in range(len(angles)):
    _a0_, _a1_ = angles[i], angles[(i+1) % len(angles)]
    if i == len(angles)-1: _a1_ += 2*pi
    _a_diff_ = _a1_ - _a0_ - 2*angle_buffer
    if _a_diff_ > angle_buffer:
        _a0_buffered_     = _a0_ + angle_buffer
        _a1_buffered_     = _a1_ - angle_buffer
        _circumference_   = 2.0 * pi * r * (_a1_buffered_ - _a0_buffered_)/(2*pi)
        cleared_angles.append((_a0_buffered_, _a1_buffered_, _circumference_))
        circumference_sum += _circumference_

if circumference_sum > 0.0:
    # Allocate points to each of the arc segments
    pts_w_arcs, _pts_left_ = [], num_of_pts
    for i in range(len(cleared_angles)):
        _a0_, _a1_, _circ_  = cleared_angles[i]
        if i == len(cleared_angles)-1: _pts_to_allocate_ = _pts_left_
        else:                          _pts_to_allocate_ = int(num_of_pts * _circ_ / circumference_sum)
        _pts_left_ -= _pts_to_allocate_
        pts_w_arcs.append(_pts_to_allocate_)

    # Plot the points
    for i in range(len(cleared_angles)):
        _a0_, _a1_, _circ_ = cleared_angles[i]
        pts_on_segment = pts_w_arcs[i]
        if   pts_on_segment == 1:
            _angle_ = (_a0_ + _a1_) / 2.0
            svg.append(f'<circle cx="{_xy_[0]+r*cos(_angle_)}" cy="{_xy_[1]+r*sin(_angle_)}" r="2" fill="#ff0000" stroke="none" />')
        elif pts_on_segment == 2:
            _angle_ = (_a0_ + _a1_) / 2.0 - (_a0_ - _a1_) / 4.0
            svg.append(f'<circle cx="{_xy_[0]+r*cos(_angle_)}" cy="{_xy_[1]+r*sin(_angle_)}" r="2" fill="#ff0000" stroke="none" />')
            _angle_ = (_a0_ + _a1_) / 2.0 + (_a0_ - _a1_) / 4.0
            svg.append(f'<circle cx="{_xy_[0]+r*cos(_angle_)}" cy="{_xy_[1]+r*sin(_angle_)}" r="2" fill="#ff0000" stroke="none" />')
        else:
            _angle_inc_ = (_a1_ - _a0_) / (pts_on_segment - 1)
            for j in range(pts_on_segment):
                _angle_ = _a0_ + _angle_inc_ * j
                svg.append(f'<circle cx="{_xy_[0]+r*cos(_angle_)}" cy="{_xy_[1]+r*sin(_angle_)}" r="2" fill="#ff0000" stroke="none" />')

else: # just dump them around the circle
    _angle_inc_ = 2.0 * pi / num_of_pts
    for i in range(num_of_pts):
        svg.append(f'<circle cx="{_xy_[0]+r*cos(_angle_inc_*i)}" cy="{_xy_[1]+r*sin(_angle_inc_*i)}" r="2" fill="#ff0000" stroke="none" />')

def radToDeg(_rad_): return _rad_ * 180.0 / pi
for _arc_ in cleared_angles:
    _a0_, _a1_, _circ_ = radToDeg(_arc_[0]), radToDeg(_arc_[1]), _arc_[2]
    svg.append(f'<path d=\"{rt.genericArc(_xy_[0], _xy_[1], _a0_, _a1_, r-5, r+5)}\" fill=\"#000000\" fill-opacity="0.1" stroke=\"black\" stroke-width=\"0.2\" />')

svg_ftr = '</svg>'
rt.tile([svg_hdr+''.join(svg)+svg_ftr])